#  Steps to take beforehand
1. Unzip reddit2012_2019.zip into directly
2. Install gpt-2-simple for finetuning gpt. pip3 install gpt-2-simple
3. Make sure to install Tensorflow 1. It doesn't work on tensorflow 2.



# concat all dataframes


In [1]:
import pandas as pd


In [20]:
!ls

10				  run3_results	      run_no_clean_ccb
2012_2016.csv			  run3_walao	      run_no_clean_china
2017.csv			  run_long_aiya       run_no_clean_gay
2018.csv			  run_long_bt4k       run_no_clean_knn
2018_clean.csv			  run_long_ccb	      run_no_clean_lah
2019.csv			  run_long_china      run_no_clean_pap
GPT - 2 model training.ipynb	  run_long_copypaste  run_no_clean_pmd
checkpoint			  run_long_gay	      run_no_clean_singlish
csv_encoded.txt			  run_long_james      run_no_clean_smlj
gen				  run_long_knn	      run_no_clean_walao
gpt2_gentext_20191113_133928.txt  run_long_lah	      run_no_clean_yishun
main.py				  run_long_pap	      samples
models				  run_long_pmd	      schedule.sh
reddit2012_2019.zip		  run_long_singapore  schedule2.sh
run3_ccb			  run_long_singlish   schedule_long.sh
run3_knn			  run_long_smlj       text_encoded.npz
run3_ns				  run_long_walao      train_df.csv
run3_nus			  run_long_yishun     try2.csv
run3_pap			  run_no_clean	      try3.csv
run3_r				  run_no_clean_ai

In [3]:
df = pd.read_csv("2012_2016.csv")

/ldap_home/james.chua/miniconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df2=pd.read_csv("2017.csv")
df3=pd.read_csv("2018.csv")
df4=pd.read_csv("2019.csv")



/ldap_home/james.chua/miniconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
all_df = pd.concat([df,df2,df3,df4]).reset_index()

In [7]:
all_df.body[1]

"Don't pretend that's not a thing you do in Singapore... I've seen 2 photographic examples already."

In [8]:
len(all_df)

3515857

In [9]:
#remove [removed]
all_df = all_df[all_df.body != "[removed]"].reset_index()

In [10]:
all_df = all_df.dropna()

In [11]:
len(all_df)

2992653

## Shuffle and replace some encoding errors

In [12]:
all_df = all_df.sample(frac=1).reset_index(drop=True)

In [13]:
all_df

,level_0,index,author,body,created_utc,date
0,1426256,174318,reddumpling,Use English to converse all the time (except e...,1493474831,2017-04-29 19:00:00
1,2002658,119191,Neralo,"&gt; These can only be done on headphones, not...",1519722453,2018-02-27 17:00:00
2,1268789,16851,asilentfilm9,Wait one more day.,1484299223,2017-01-13 06:00:00
3,2130345,246878,sraelgaiznaer,What’s with the 3D version? Isn’t it good enou...,1523704828,2018-04-14 19:00:00
4,1234346,1234346,shqippotato,dont u know losing internet connectivity can l...,1480854504,2016-12-04 09:00:00
...,...,...,...,...,...,...
2992648,3136284,393654,WalfAkaiTsuki,Yesterday my wallet dropped out of my pocket a...,1560216032,2019-06-11 06:00:00
2992649,2835463,92833,dontknowla,Audio mixing you need to be a chio bu then con...,1549463366,2019-02-06 22:00:00
2992650,3237747,495117,cmonplsnotnow,I’m just wondering how people love that stuff ...,1563604484,2019-07-20 14:00:00
2992651,1506995,255057,Hurt_cow,thanks,1498098367,2017-06-22 06:00:00


In [18]:
# Save to csv called train_df

In [19]:
all_df[(all_df.body.str.len() > 100)].to_csv("train_df.csv",index=False)

# Run the finetuning
You can change the model_name to "124M" for faster training and less memory requirements but lower quality results.
We required 50gb of RAM for the 355M model

In [24]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/355M/

sess = gpt2.start_tf_sess()
csv_path = "train_df.csv"
gpt2.encode_csv(csv_path=csv_path, out_path="csv_encoded.txt")
gpt2.encode_dataset(file_path="csv_encoded.txt", out_path="text_encoded.npz")
gpt2.finetune(sess,
            dataset="text_encoded.npz",
            model_name=model_name,
            steps=8000,
            run_name='run_long',
            print_every=10,
            sample_every=200,
            save_every=500,
            overwrite=True
            )

# Run inference
gpt_2_simple provides a command line interface
Some examples:

!gpt_2_simple generate --temperature 0.8 --nsamples 80 --batch_size 8 --prefix "In NUS, a group of students took the module BT4222, Mining Web Data for Business Insights, under Assistant Professor Qiao Dandan. The students had to create a project using Natural Language Processing tools. One of her students, James, did not know what to do for his project and so" --truncate "<|endoftext|>" --nfiles 5 --folder run_long_james --run_name run_long

!gpt_2_simple generate --temperature 0.8 --nsamples 80 --batch_size 8 --prefix "SINGAPORE" --truncate "<|endoftext|>" --nfiles 5 --folder run_long_singapore --run_name run_long

!gpt_2_simple generate --temperature 0.8 --nsamples 80 --batch_size 8 --prefix "PAP" --truncate "<|endoftext|>" --nfiles 5 --folder run_long_pap --run_name run_long